In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library


## Gather Table form wiki

In [78]:
from bs4 import BeautifulSoup as bs
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
source = requests.get(url).text
soup = BeautifulSoup(source,'html.parser')
table= soup.find('table')
toronto_df = pd.read_html(str(table))[0]
toronto_df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Remove Non assigned Value

In [83]:
#Remove Borough not assigned value
df.drop(df[df['Borough'] == 'Not assigned' ].index, inplace=True)
#Remove Neighbourhood not assigned value
df.drop(df[df['Neighbourhood'] == 'Not assigned' ].index, inplace=True)
#show Data Frame
df

,PostalCode,Borough,Neighbourhood
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Import coordinates data

In [100]:
Codi = pd.read_csv('https://cocl.us/Geospatial_data')
Codi.rename(columns={'Postal Code':'PostalCode'},inplace=True)
Codi.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge Table

In [102]:
data_Table = pd.merge(df, Codi)
data_Table 

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4A,North York,Victoria Village,43.725882,-79.315572
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
4,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
...,...,...,...,...,...
97,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
98,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
99,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
100,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [132]:
Toronto_Table = data_Table[data_Table ['Borough'].str.contains('Toronto',regex=False)]
Toronto_Table.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
18,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [115]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(Toronto_Table['Latitude'],Toronto_Table['Longitude'],Toronto_Table['Borough'],Toronto_Table['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [133]:
from sklearn.cluster import KMeans
k=5
toronto_clustering = Toronto_Table.drop(['PostalCode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
Toronto_Table.insert(0,'CLabels', kmeans.labels_)

In [128]:
Toronto_Table

,CLabels,PostalCode,Borough,Neighbourhood,Latitude,Longitude
1,3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
8,3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
18,2,M4E,East Toronto,The Beaches,43.676357,-79.293031
19,3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
23,3,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
24,4,M6G,Downtown Toronto,Christie,43.669542,-79.422564
29,3,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
30,4,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
